<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#RandomForestClassifier-(с-подбором-гиперпараметров)" data-toc-modified-id="RandomForestClassifier-(с-подбором-гиперпараметров)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>RandomForestClassifier (с подбором гиперпараметров)</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Начнем новый проект:

- Импортируем все необходимые библиотеки

- Проведем небольшой анализ

- Преобразуем комментарии в корпус, проведем лемматизацию или стемминг

- Далее рассмотрим 2-3 модели (попытаюсь и BERT, но не уверен в мощности своего компьютера)

- В моделях будем пробовать различные гиперпараметры и возможно сбалансируем классы

In [1]:
import pandas as pd
import numpy as np
import torch
import transformers
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as stopwords_nltk
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

Прочитаем файл и посмотрим необходимую информацию

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

Выведем первые 5 строк, посмотрим инфо и распределение

In [3]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Имеем 2 столбца по 159571 строке - текст и определение токсичности (0 - обычный комментарий, 1 - токсичный)

In [5]:
data.describe()

,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Посмотрим на соотношение количества обычных комментариев и токсичных

In [6]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

По количеству наблюдаем некий дисбаланс классов - токсичных комментариев очевидно меньше, чем обычных

Посмортим по моделям - возможно имеет место быть дисбаланс классов

Посмотрим что система принимает за токсичный комментарий

In [7]:
data[data['toxic'] == 1].head()

,text,toxic
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
12,Hey... what is it..\n@ | talk .\nWhat is it......,1
16,"Bye! \n\nDon't look, come or think of comming ...",1
42,You are gay or antisemmitian? \n\nArchangel WH...,1
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1


Уберем все ненужные символы и оставим сухой текст

Далее проведем лемматизацию текста и посмотрим что получилось в итоге

In [8]:
def clear_text_and_lemm(text):
    wn1 = WordNetLemmatizer()
    sub = re.sub(r'[^a-zA-Z]',' ', text)
    lemm = nltk.word_tokenize(sub)
    sup =" ".join([wn1.lemmatize(words) for words in lemm])
    return sup

Применим нашу очистку и лемматизацию и применим функцию для дальнейшей работы

In [9]:
data['lemm_text'] = data['text'].apply(clear_text_and_lemm)

In [10]:
data['lemm_text'] = data['lemm_text'].str.lower()

In [11]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


### Вывод:

- Провели первичное ислледование

- Оставили только регулярные выражения и провели лемматизацю

- Создали корпус постов

- создали мешок слов

## Обучение моделей

На данном этапе произведем следующие действия:
    
- Выделим таргет и фичи

- Разделим данные на 2 выборки (выделим 25% для тестовой выборки)

- Уберем стоп слова и переведем наши данные в Unicode для дальнейшей работы

- протестируем 2 модели - Логистическую Классификацию и Случайный лес


In [12]:
target = data['toxic']
features = data.drop(['toxic','text'], axis = 1)

print('Целевой признак:', target.shape)
print('Признаки:', features.shape)

Целевой признак: (159571,)
Признаки: (159571, 1)


In [13]:
features_train, features_test, target_train, target_test = train_test_split(features,
                                                                            target,
                                                                            test_size=0.25,
                                                                           stratify = target) 
                                                                            

In [14]:
print('Размер обучающей выборки:', features_train.shape,
      
     'Размер тестовой выборки:', features_test.shape)

Размер обучающей выборки: (119678, 1) Размер тестовой выборки: (39893, 1)


In [15]:
corpus_train = features_train['lemm_text']
corpus_test = features_test['lemm_text']

Избавимся от стоп-слов и сделаем векторизацию

In [16]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
train_X = count_tf_idf.fit_transform(corpus_train)
test_X = count_tf_idf.transform(corpus_test)
print(train_X.shape)
print(test_X.shape)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


(119678, 138366)
(39893, 138366)


### LogisticRegression

Первую модель возьмем логистическую регрессию - сразу в настройках сбалансируем классы (у нас примерно 10% токсичных твитов от общего числа записей)

In [17]:
model=LogisticRegression(random_state=12345, class_weight='balanced', C = 10)

In [18]:
%%time
model.fit(train_X, target_train)

CPU times: user 21.7 s, sys: 27.7 s, total: 49.4 s
Wall time: 49.5 s


LogisticRegression(C=10, class_weight='balanced', random_state=12345)

In [19]:
%%time
predictions = model.predict(test_X)

CPU times: user 0 ns, sys: 5.54 ms, total: 5.54 ms
Wall time: 4.2 ms


In [20]:
f1 = f1_score(target_test, predictions)
print('LogisticRegression:', f1)

LogisticRegression: 0.757734303912648


Модель показала отличный результат - посмотрим на Слуйчаный лес

### RandomForestClassifier (с подбором гиперпараметров)

In [21]:
model = RandomForestClassifier(class_weight = 'balanced')
parametrs = { 'n_estimators': range (10, 51, 10),
              'max_depth': range (10, 51, 10) }
grid = GridSearchCV(estimator=model, cv=5,
                        param_grid=parametrs, verbose=2)
grid.fit(train_X, target_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] END ......................max_depth=10, n_estimators=10; total time=   1.8s
[CV] END ......................max_depth=10, n_estimators=10; total time=   1.8s
[CV] END ......................max_depth=10, n_estimators=10; total time=   1.8s
[CV] END ......................max_depth=10, n_estimators=10; total time=   1.8s
[CV] END ......................max_depth=10, n_estimators=10; total time=   1.8s
[CV] END ......................max_depth=10, n_estimators=20; total time=   3.5s
[CV] END ......................max_depth=10, n_estimators=20; total time=   3.8s
[CV] END ......................max_depth=10, n_estimators=20; total time=   3.7s
[CV] END ......................max_depth=10, n_estimators=20; total time=   3.6s
[CV] END ......................max_depth=10, n_estimators=20; total time=   3.6s
[CV] END ......................max_depth=10, n_estimators=30; total time=   5.2s
[CV] END ......................max_depth=10, n_

GridSearchCV(cv=5, estimator=RandomForestClassifier(class_weight='balanced'),
             param_grid={'max_depth': range(10, 51, 10),
                         'n_estimators': range(10, 51, 10)},
             verbose=2)

In [22]:
print(grid.best_params_)

{'max_depth': 50, 'n_estimators': 40}


In [23]:
model = RandomForestClassifier(max_depth= 50, n_estimators= 50, class_weight = 'balanced', random_state=12345)

In [24]:
%%time
model.fit(train_X, target_train) 

CPU times: user 22.7 s, sys: 52.5 ms, total: 22.7 s
Wall time: 22.7 s


RandomForestClassifier(class_weight='balanced', max_depth=50, n_estimators=50,
                       random_state=12345)

In [25]:
%%time
predictions = model.predict(test_X)

CPU times: user 540 ms, sys: 68 µs, total: 541 ms
Wall time: 546 ms


In [26]:
f1 = f1_score(target_test, predictions)
print('RandomForestClassifier:', f1)

RandomForestClassifier: 0.4644816847100696


Очень низкий показатель модели - у Логистической регрессии было гораздо лучше

## Выводы

В данном проекте были выволнены следующие шаги:
    
- Просмотрен файл с записями и указанием их токсичности

- выполнена очистка и лемматизация для данного текста

- слова переведены в корпус, очищены от ненужных слов и подготовлены к работе с моделями

- произведено исследование с 2 моделями - Логистической регрессией и Случайным лесом с подбором гиперпараметров

- в результате исследования лучший результат показала модель - 0.75044 (Логистическая регрессиия)